In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [3]:
news_df = pd.DataFrame({'document':documents})

In [4]:
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [5]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어 제거

In [6]:
from gensim import corpora

In [7]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


In [8]:
print(dictionary[89])

need


In [9]:
len(dictionary)

181856

In [10]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.004*"would" + 0.004*"also" + 0.004*"government" + 0.003*"used"')
(1, '0.003*"cincinnati" + 0.003*"price:" + 0.002*"title:" + 0.002*"conductor"')
(2, '0.007*"please" + 0.007*"using" + 0.006*"also" + 0.006*"available"')
(3, '0.026*"armenian" + 0.020*"armenians" + 0.020*"turkish" + 0.009*"turkey"')
(4, '0.021*"file" + 0.015*"entry" + 0.011*"chip" + 0.009*"----------------------------------------------------------------------"')
(5, '0.008*"henrik]" + 0.005*"bytes" + 0.004*"candida" + 0.004*"braves"')
(6, '0.015*"55.0" + 0.003*"0.333" + 0.002*"creed" + 0.002*"month,"')
(7, '0.007*"scsi-2" + 0.006*"scsi-1" + 0.004*"5.25"" + 0.004*"3.5""')
(8, '0.012*"period" + 0.009*"power" + 0.008*"---------------" + 0.007*"scorer"')
(9, '0.004*"john\'s" + 0.003*"--------------------------------------------------------------------------------" + 0.003*"rochester" + 0.003*"canadiens"')
(10, '0.016*"file:" + 0.004*"bikes" + 0.004*"u.s.a." + 0.003*"yankees"')
(11, '0.013*"greek" + 0.013*"----" + 0.007*

In [11]:
print(ldamodel.print_topics())

[(0, '0.004*"would" + 0.004*"also" + 0.004*"government" + 0.003*"used" + 0.003*"number" + 0.003*"many" + 0.003*"public" + 0.003*"system" + 0.003*"first" + 0.003*"people"'), (1, '0.003*"cincinnati" + 0.003*"price:" + 0.002*"title:" + 0.002*"conductor" + 0.002*"chipset" + 0.002*"(301)" + 0.002*"then?" + 0.002*"documentary" + 0.002*"yamaha" + 0.002*"oakland"'), (2, '0.007*"please" + 0.007*"using" + 0.006*"also" + 0.006*"available" + 0.006*"windows" + 0.006*"anyone" + 0.005*"like" + 0.005*"file" + 0.005*"software" + 0.005*"would"'), (3, '0.026*"armenian" + 0.020*"armenians" + 0.020*"turkish" + 0.009*"turkey" + 0.008*"turks" + 0.007*"armenia" + 0.007*"genocide" + 0.006*"russian" + 0.006*"killed" + 0.005*"muslim"'), (4, '0.021*"file" + 0.015*"entry" + 0.011*"chip" + 0.009*"----------------------------------------------------------------------" + 0.008*"keys" + 0.008*"char" + 0.008*"clipper" + 0.008*"algorithm" + 0.008*"encryption" + 0.007*"output"'), (5, '0.008*"henrik]" + 0.005*"bytes" + 0.

In [12]:
import pyLDAvis.gensim_models

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [14]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.24575335), (5, 0.18947078), (7, 0.015455226), (13, 0.37482342), (19, 0.16293043)]
1 번째 문서의 topic 비율은 [(0, 0.09334036), (6, 0.025380246), (13, 0.32548338), (14, 0.03675518), (19, 0.50064117)]
2 번째 문서의 topic 비율은 [(0, 0.06938), (3, 0.03744047), (5, 0.09805459), (13, 0.46354648), (19, 0.31923583)]
3 번째 문서의 topic 비율은 [(0, 0.6397185), (4, 0.03582404), (13, 0.018136634), (19, 0.29430172)]
4 번째 문서의 topic 비율은 [(4, 0.10047641), (5, 0.22419176), (10, 0.07578721), (14, 0.19601461), (19, 0.37571913)]


In [15]:

def make_topictable_per_doc(ldamodel, corpus):
    topic_data = []


    for i, topic_list in enumerate(ldamodel[corpus]):
        # 'per_word_topics' 매개변수에 따라 문서의 토픽 분포를 추출
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list

        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                # 가장 비중이 높은 토픽에 대해 문서 번호, 토픽 번호, 비중, 전체 토픽 리스트를 추가
                topic_data.append([int(topic_num), round(prop_topic, 4), topic_list])
                break  # 가장 높은 비중의 토픽만 필요합니다.

    # 결과 리스트를 pandas DataFrame으로 변환
    topic_df = pd.DataFrame(topic_data, columns=['가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])

    return topic_df

In [16]:
topictable = make_topictable_per_doc(ldamodel, corpus)

In [17]:
topictable.columns = ['가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']

In [18]:
topictable[:10]

,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,13,0.3748,"[(0, 0.24575211), (5, 0.1894709), (7, 0.015455..."
1,19,0.5007,"[(0, 0.09334291), (6, 0.025380254), (13, 0.325..."
2,13,0.4635,"[(0, 0.069451585), (3, 0.037439164), (5, 0.098..."
3,0,0.6397,"[(0, 0.63970304), (4, 0.0358243), (13, 0.01816..."
4,19,0.3757,"[(4, 0.100504555), (5, 0.22419024), (10, 0.075..."
5,13,0.3424,"[(3, 0.31968954), (13, 0.3423805), (18, 0.0472..."
6,2,0.5615,"[(0, 0.055686843), (2, 0.56149906), (19, 0.371..."
7,19,0.4637,"[(0, 0.035707455), (10, 0.06644611), (13, 0.22..."
8,0,0.5591,"[(0, 0.55909437), (4, 0.21114208), (12, 0.0276..."
9,19,0.5533,"[(0, 0.18879679), (2, 0.16563864), (11, 0.0577..."
